A0(Administered activity (MBq))

t (time)

tU	Uptake time (h)

tI	Imaging time (h)

D(t)	Total dose for time t(µSv)

D(0)	Initial dose rate (µSv/ h)

T1/2	Radionuclide half-life (h)

Rt	Dose reduction factor over time t	=1.443X(T1/2 / t) X[1 −exp(−0.693t / T1/2)]

RtU Dose reduction factor over uptake time at time t  =1.443X(T1/2 / tU) X(1 −exp(−0.693tU / T1/2))

RtI Dose reduction factor over imaging time           =1.443X(T1/2 / tI) X[1 −exp(−0.693tI / T1/2)]

Nw	Number of patients per week

d	Distance from source to barrier (m)

FU	Uptake time decay factor (µSv)	=exp[ − 0.693(tU / T1/2)] 

T		Occupancy factor

P	Weekly dose limit

B Transmission factor (for Uptake room =10.9X P X d2 / [T X Nw X Ao X tU(h) X RtU]) (for scan room =12.8 X P X d2 / [T X Nw X Ao X FU X tI(h) X RtI])



http://icrpaedia.org/Dose_limits

In [21]:
import math

# Constants
A0 = 555  # Administered activity (MBq)
T = 1  # Occupancy factor
T1_2 = 109.8 / 60  # Radionuclide half-life in hours (109.8 minutes)
dr = 0.092  # Dose rate immediately after administration (µSv m²/MBq h)
P_controlled = 100  # Weekly dose limit for controlled areas (µSv)
P_uncontrolled = 20  # Weekly dose limit for uncontrolled areas (µSv)

# Function to get user input for variables
def get_inputs():
    Nw = int(input("Enter Number of patients per week: "))
    d = float(input("Enter Distance from source to barrier (m): "))
    tU = float(input("Enter Uptake time (hours): "))
    tI = float(input("Enter Scan time (hours): "))

    return Nw, d, tU, tI



# Function to calculate the ***dose reduction factor*** over uptake time
def RtU(T1_2, tU):
    return 1.443 * (T1_2 / tU) * (1 - math.exp(-0.693 * tU / T1_2))


# Function to calculate the dose reduction factor over imaging time
def RtI(T1_2, tI):
    return 1.443 * (T1_2 / tI) * (1 - math.exp(-0.693 * tI / T1_2))


#The total dose at a point d meters from the patient during the uptake time(tU) is
def DtU (dr, A0, tU,RtU, d):
    return dr * A0 * tU * (RtU/d**2)


# Function to calculate transmission factor for the uptake room
def B_uptake_room(P, d, T, Nw, A0, tU, RtU):
    return 10.9 * P * (d ** 2) / (T * Nw * A0 * tU * RtU)

def Fu(tU):
    return math.exp (0.693 * (tU*60) /110)

# Function to calculate transmission factor for the Scan room
def B_scan_room(P, d, T, Nw, A0, tI, RtI):
    return 12.8 * P * (d ** 2) / (T * Nw * A0 * 0.85 * tI * Fu* RtI)




# Main function to run the calculation for both controlled and uncontrolled areas
def calculate_transmission_factors():
    # Get user inputs
    Nw, d, tU, tI = get_inputs()
    
    # Calculate the dose reduction factor RtU
    rtU = RtU(T1_2, tU)
    
    # Calculate the transmission factor B for uncontrolled and controlled areas
    B_U_uncontrolled = B_uptake_room(P_uncontrolled, d, T, Nw, A0, tU, rtU)
    B_U_controlled = B_uptake_room(P_controlled, d, T, Nw, A0, tU, rtU)
    B_S_uncontrolled = B_scan_room(P_uncontrolled, d, T, Nw, A0, tU, rtU)
    B_S_controlled = B_scan_room(P_controlled, d, T, Nw, A0, tI, rtU)

    
    # Output the results
    print(f"\nTransmission Factor (B) for the uptake room (Uncontrolled area): {B_uncontrolled:.4f}")
    print(f"Transmission Factor (B) for the uptake room (Controlled area): {B_controlled:.4f}")
    print(f"\nTransmission Factor (B) for the Scan room (Uncontrolled area): {B_uncontrolled_imaging:.4f}")
    print(f"Transmission Factor (B) for the Scan room (Controlled area): {B_controlled_imaging:.4f}")

# Run the calculations
calculate_transmission_factors()


ValueError: invalid literal for int() with base 10: ''

In [15]:
# Constants for Imaging room
void_fraction = 0.85  # 15% of the activity is voided, so 85% remains
FU = math.exp(-0.693 * 60 / 110)  # Decay factor for F-18 after 1 hour (FU = 0.68)
# Function to get user input for variables
def get_inputs():
    Nw = int(input("Enter Number of patients per week: "))
    d = float(input("Enter Distance from source to barrier (m): "))
    

    return Nw, d, tI

# Function to calculate the dose reduction factor over imaging time
def RtI(T1_2, tI):
    return 1.443 * (T1_2 / tI) * (1 - math.exp(-0.693 * tI / T1_2))

# Function to calculate transmission factor for the imaging room
def B_imaging_room(P, d, T, Nw, A0, FU, void_fraction, tI, RtI):
    return 12.8 * P * (d ** 2) / (T * Nw * A0 * void_fraction * FU * tI * RtI)

# Main function to calculate transmission factors for the imaging room
def calculate_transmission_factors_imaging_room():
    # Get user inputs for imaging time
    Nw, d, tI = get_inputs()  # Reuse the same input method from previous step
    
    # Calculate the dose reduction factor RtI
    rtI = RtI(T1_2, tI)
    
    # Calculate the transmission factor B for uncontrolled and controlled areas in imaging room
    B_uncontrolled_imaging = B_imaging_room(P_uncontrolled, d, T, Nw, A0, FU, void_fraction, tI, rtI)
    B_controlled_imaging = B_imaging_room(P_controlled, d, T, Nw, A0, FU, void_fraction, tI, rtI)
    
    # Output the results for imaging room
    print(f"\nTransmission Factor (B) for the imaging room (Uncontrolled area): {B_uncontrolled_imaging:.4f}")
    print(f"Transmission Factor (B) for the imaging room (Controlled area): {B_controlled_imaging:.4f}")

# Run the calculations for the imaging room
calculate_transmission_factors_imaging_room()



Transmission Factor (B) for the imaging room (Uncontrolled area): 0.3912
Transmission Factor (B) for the imaging room (Controlled area): 1.9559
